In [1]:
"""Basic demonstration of sweeps and metrics operation."""

# %%
# Imports, etc.
import pickle

import numpy as np
from functools import partial
import torch


from transformer_lens import HookedTransformer

from activation_additions import (
    logits,
    hyperparameter_search,
    prompt_utils,
    utils,
    metrics,
    hook_utils
)

utils.enable_ipython_reload()

# Disable gradients to save memory during inference
_ = torch.set_grad_enabled(False)

from copy import deepcopy
from typing import List, Union,Dict
import pandas as pd
from transformer_lens.utils import get_act_name

In [2]:
# %%
# Load a model
MODEL = HookedTransformer.from_pretrained(model_name="gpt2-xl", device="cpu")
_ = MODEL.to("cuda:0")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda:0


In [39]:
def conditional_perplexity(model,prompt_tokens,completion_tokens):
    metric_func=metrics.get_logprob_metric(model)
    completed_tokens=torch.cat((prompt_tokens, completion_tokens), dim=1)
    metric=metric_func([completed_tokens])
    completion_logprobs=metric["logprob_actual_next_token"].array[0][-completion_tokens.shape[1]:]
    return -sum(completion_logprobs)

In [36]:
perplexity,  metric, completed_tokens=conditional_perplexity(MODEL,
    MODEL.to_tokens("The only thing we have to fear is"),MODEL.to_tokens(" real fear.")[:, 1:])
perplexity

9.406886041164398

In [45]:
list(range(0,10,2))

[0, 2, 4, 6, 8]

In [49]:
grid_df=layer_coefficient_gridsearch(
    model=MODEL,
    prompts=["The Most beautyful city in the world is"],
    weighted_steering_prompts={" Rome":1," Paris":-1},
    Layer_list=list(range(6,12)),
    coefficient_list=list(range(0,10,2)),
    wanted_completions=" Rome",
    unwanted_completions=" Paris")

TypeError: 'list' object is not callable

In [48]:
def layer_coefficient_gridsearch(
    model: HookedTransformer,
    prompts: Union[str, List[str]],
    weighted_steering_prompts: Dict[str, float],
    Layer_list: List[int],
    coefficient_list: List[float],
    wanted_completions: Union[str, List[str]],
    unwanted_completions: Union[str, List[str]],
) -> pd.DataFrame:

    metric_func=metrics.get_logprob_metric(model)

    prompt_tokens=[model.to_tokens(prompt)for prompt in prompts]
    wanted_completion_tokens=[model.to_tokens(wanted_completion)[:, 1:] for wanted_completion in wanted_completions]
    unwanted_completion_tokens=[model.to_tokens(unwanted_completion)[:, 1:] for unwanted_completion in unwanted_completions]

    layer_data = []
    coefficient_data = []
    perplexity_wanted_data = []
    perplexity_unwanted_data = []

    for layer in Layer_list:
        for coefficient in coefficient_list:
            ActAds =[prompt_utils.ActivationAddition(
                        coeff=prompt_weighting*coefficient,
                        act_name=layer,
                        prompt=prompt) for prompt, prompt_weighting in weighted_steering_prompts.items()]
            hook_fns=hook_utils.hook_fns_from_activation_additions(model, ActAds)
            
            for act_name, hook_fn in hook_fns.items():
                model.add_hook(act_name, hook_fn)
                
            perplexity_on_wanted=[conditional_perplexity(model, prompt, completion) for prompt, completion in zip(prompt_tokens, wanted_completion_tokens)]
            perplexity_on_unwanted=[conditional_perplexity(model, prompt, completion) for prompt, completion in zip(prompt_tokens, unwanted_completion_tokens)]
            
            model.remove_all_hook_fns()

            # Append data for this layer and coefficient to the lists
            layer_data.extend([layer] * len(prompts))
            coefficient_data.extend([coefficient] * len(prompts))
            perplexity_wanted_data.extend(perplexity_on_wanted)
            perplexity_unwanted_data.extend(perplexity_on_unwanted)

    # Create DataFrame
    df = pd.DataFrame({
        "Layer": layer_data,
        "Coefficient": coefficient_data,
        "Perplexity (wanted)": perplexity_wanted_data,
        "Perplexity (unwanted)": perplexity_unwanted_data,
        "prompts": prompts,
        "wanted_completions":wanted_completions,
        "unwanted_completions":unwanted_completions
    })

    return df


In [49]:
steering_vector=prompt_utils.weighted_prompt_superposition(MODEL,{"hello":1.2,"bye":-1.2})

In [60]:
ActAds =[prompt_utils.ActivationAddition(coeff=prompt_weighting*coefficient, act_name=layer,prompt=prompt) for prompt, prompt_weighting in weighted_steering_prompts.items()]
hook_fns=hook_utils.hook_fns_from_activation_additions(model, ActAds)

NameError: name 'weighted_steering_prompts' is not defined

In [61]:
ActAds =[prompt_utils.ActivationAddition(coeff=1, act_name=1,prompt="hi"),
         prompt_utils.ActivationAddition(coeff=-1, act_name=1,prompt="bye")]
hook_fns=hook_utils.hook_fns_from_activation_additions(MODEL, ActAds)

TypeError: 'list' object is not callable

In [52]:
ActAds=prompt_utils.ActivationAddition(coeff=1, act_name=1,prompt="hi")
hook_utils.hook_fns_from_activation_additions(MODEL, [ActAds])

TypeError: 'list' object is not callable

In [57]:
layers=list(range(20))
coefficents=list(range(20))
activation_additions=[]

for layer in layers:
    vector_layer=get_x_vector_preset(prompt1=" love", prompt2=" hate", coeff=1, act_name=layer)
    for coefficent in coefficents:
        vector=deepcopy(vector_layer)
        for i in range(len(vector)):
            vector[i].coeff=coefficent*vector[i].coeff
        activation_additions.append(vector)


In [64]:
activation_additions[2]
logits.get_token_probs(MODEL,prompts,activation_additions[1],return_positions_above)

probs                                                                \
        0         1             2             3         4             5       
pos                                                                           
5    0.000006  0.000007  4.937604e-07  2.832087e-07  0.000001  3.916537e-07   

                                             ...   logprobs             \
        6         7         8         9      ...      50247      50248   
pos                                          ...                         
5    0.000006  0.000007  0.000008  0.000004  ... -15.978655 -15.196625   

                                                                      \
         50249      50250      50251     50252      50253      50254   
pos                                                                    
5   -18.414135 -14.930026 -17.293758 -15.18249 -14.840092 -16.476681   

                           
         50255      50256  
pos                        
5   -15.081723 -11.901134  

[1 rows x 100514 columns]

In [75]:

get_x_vector_preset = partial(prompt_utils.get_x_vector, pad_method="tokens_right",
                              model=MODEL,
                              custom_pad_id=MODEL.to_single_token(" "))
activation_additions=get_x_vector_preset(prompt1=" love", prompt2=" hate", coeff=1, act_name=6)
prompts="I love you. The"
return_positions_above=len(MODEL.tokenizer.encode(prompts))
matrix=[]
for activation_addition in [activation_additions]:
    print(8)
    a=logits.get_token_probs(MODEL,prompts,activation_addition,return_positions_above)

8


In [56]:
matrix

[        probs                                                                \
         0         1             2             3         4             5       
 pos                                                                           
 5    0.000005  0.000007  4.419383e-07  2.559607e-07  0.000001  3.399807e-07   
 
                                              ...   logprobs             \
         6         7         8         9      ...      50247      50248   
 pos                                          ...                         
 5    0.000006  0.000006  0.000008  0.000004  ... -16.132639 -15.316798   
 
                                                                        \
          50249      50250      50251      50252      50253      50254   
 pos                                                                     
 5   -18.390577 -14.896774 -17.277262 -15.282727 -14.884817 -16.464771   
 
                            
          50255      50256  
 pos                

In [31]:
activation_additions[0]

ActivationAddition(tensor([50256,  1842], device='cuda:0'), 1, blocks.6.hook_resid_pre)

In [34]:
print(dir(activation_additions[0]))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'act_name', 'coeff', 'tokens']


In [36]:
activation_additions[0]=2